<div align="center" style="direction:rtl; line-height:300%;">
    <font face="B Nazanin" size="12"><b>به نام خدا<b></font>
    <br>
    <img src="logo.png" alt="لوگو" style="width:250px; height:auto;">
    <br>
    <font face="B Nazanin" size="5"><b>دانشگاه صنعتی شریف</b></font>
    <br><br>
    <font face="B Nazanin" size="5"><b>بازشناسی گفتار<b></font>
    <br><br>
    <font face="B Nazanin" size="5"><b>استاد: جناب آقای دکتر صامتی</b></font>
    <br><br>
    <font face="B Nazanin" size="5">دانشجویان: محمد شیرخانی، علی بابابیگ</font>
    <br><br>
    <font face="B Nazanin" size="5">زمستان 1403</font>
    <br><br>
    <hr>
    <font face="B Nazanin" size="5">پروژه پایانی</font>
</div>

<div style="direction:rtl; line-height:200%; font-family: 'B Nazanin', serif; font-size: 18px; text-align: justify;">
    <h3>
        1- نصب کتابخانه‌ها و وارد کردن ماژول‌ها
    </h3>
    <p>
    <div dir="rtl" style="text-align: justify;">
        در این سلول، ابتدا کتابخانه های مورد نیاز مانند hazm، torchaudio، datasets و noisereduce با استفاده از دستور pip install نصب شده‌اند. سپس، کتابخانه‌ها و ماژول‌های مورد استفاده برای پردازش متن و صدا وارد برنامه شده‌اند.
    </div>
    </p>
</div>

In [ ]:
!pip install hazm
!pip install torchaudio
!pip install datasets
!pip install noisereduce  # for spectral gating noise reduction

import re
import string
import torch
import torchaudio
import numpy as np
import IPython

from hazm import Normalizer
from datasets import load_dataset, Audio

import noisereduce as nr

<div style="direction:rtl; line-height:200%; font-family: 'B Nazanin', serif; font-size: 18px; text-align: justify;">
    <h3>
        2- پیش‌پردازش و نرمال‌سازی متن فارسی
    </h3>
    <p>
    <div dir="rtl" style="text-align: justify;">
        در این سلول ابتدا یک نرمالایزر از کتابخانه Hazm تعریف می‌شود. سپس لیستی از کاراکترهای نامطلوب و نگاشت‌هایی برای تبدیل کاراکترهای فارسی به شکل یکپارچه مشخص می‌شود. در ادامه توابعی برای جایگزینی کاراکترها (multiple_replace) و حذف کاراکترهای اضافی (remove_chars_to_ignore) نوشته شده و نهایتاً تابع normalize_persian_text، متن ورودی را با استفاده از این توابع نرمالسازی می‌کند.
    </div>
    </p>
</div>

In [ ]:
# Hazm normalizer
_normalizer = Normalizer()

# Characters to remove or ignore:
chars_to_ignore = [
    ",", "?", ".", "!", "-", ";", ":", "\"", "“", "%", "‘", "”", "�", "–", "…", "_", "”", "“", "„",
    "#", "!", "؟", "(", ")", "؛", "«", "»", "٬", "ٔ", "َ", "ْ", "ٌ", "ٍ",
    # English letters/digits (if we decide to remove them or map them):
    *list(string.ascii_lowercase + string.digits)
]

# Map some Persian variants to unified forms:
chars_to_mapping = {
    'ك': 'ک', 'دِ': 'د', 'بِ': 'ب', 'زِ': 'ز', 'ذِ': 'ذ', 'شِ': 'ش', 'سِ': 'س',
    'ى': 'ی', 'ي': 'ی', 'أ': 'ا', 'ؤ': 'و', 'ۀ': 'ه', 'ة': 'ه', 'ـ': '',
    "ﯾ": "ی", 'ﯿ': "ی", 'ﺑ': "ب", 'ﺪ': "د", 'ﺮ': "ر", 'ﺴ': "س",
    'ﺷ': "ش", 'ﺸ': "ش", 'ﻫ': "ه", 'ﻭ': "و", "ﮔ": "گ",
    "\u200c": " ", "\u200d": " ", "\u200e": " ", "\u200f": " ", "\ufeff": " ",
}

def multiple_replace(text, mapping):
    """
    Replace all occurrences of the keys in mapping with their values in text.
    """
    pattern = "|".join(map(re.escape, mapping.keys()))
    return re.sub(pattern, lambda m: mapping[m.group()], text)

def remove_chars_to_ignore(text, chars):
    """
    Remove characters defined in chars from text.
    """
    chars_regex = f"[{''.join(chars)}]"
    return re.sub(chars_regex, "", text)

def normalize_persian_text(batch):
    """
    Applies Persian text normalization using Hazm and additional cleaning.
    Expects a batch with the key 'transcription'.
    """
    raw_text = batch["transcription"]
    if not isinstance(raw_text, str):
        # In case of unexpected data format
        raw_text = str(raw_text)

    # Basic cleanup
    text = raw_text.strip().lower()

    # Hazm normalization 
    text = _normalizer.normalize(text)

    # Map Persian variants to unified forms
    text = multiple_replace(text, chars_to_mapping)

    # Remove unwanted characters
    text = remove_chars_to_ignore(text, chars_to_ignore)

    # Replace multiple spaces with single space
    text = re.sub(r"\s+", " ", text).strip()

    # Update the transcription
    batch["transcription"] = text
    return batch

<div style="direction:rtl; line-height:200%; font-family: 'B Nazanin', serif; font-size: 18px; text-align: justify;">
    <h3>
        3- پردازش و نرمال‌سازی اولیه داده‌های صوتی و متنی
    </h3>
    <p>
    <div dir="rtl" style="text-align: justify;">
        در این بخش، ابتدا دیتاست بارگذاری شده و فرکانس نمونه‌برداری صوت به ۱۶۰۰۰ هرتز تغییر می‌کند. سپس با استفاده از توابع نرمال‌سازی (Hazm به همراه نرمالسازی سفارشی) متن‌های دیتاست پاک‌سازی می‌شوند. در نهایت، نمونه‌ای از متن و صوت قبل و بعد از نرمال‌سازی به نمایش گذاشته می‌شود.
    </div>
    </p>
</div>

In [ ]:
dataset_raw = load_dataset("pourmand1376/asr-farsi-youtube-chunked-10-seconds")

# apply sampling rate conversion for all splits:
dataset_raw = dataset_raw.cast_column("audio", Audio(sampling_rate=16000))

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
# Apply text normalization functions to all splits:
dataset_pp = dataset_raw.map(
    normalize_persian_text,
    desc="Normalizing text (Hazm + custom cleanup)"
)

In [ ]:
print(f"--- Raw Sample 0 ---")
print("Raw Transcription:", dataset_raw['train'][0]["transcription"])
raw_audio_data = dataset_raw['train'][0]["audio"]["array"]
raw_audio_sr = dataset_raw['train'][0]["audio"]["sampling_rate"]
IPython.display.Audio(data=raw_audio_data, rate=raw_audio_sr)

--- Raw Sample 0 ---
Raw Transcription: بعد از اینکه دیوان عالی ایالات متحده هم سال ۱۹۲۷ این قانون رو تصویب کرد،


In [ ]:
print("Sample after text normalization:")
print(dataset_pp['train'][0]["transcription"])

Sample after text normalization:
بعد از اینکه دیوان عالی ایالات متحده هم سال ۱۹۲۷ این قانون رو تصویب کرد،


<div style="direction:rtl; line-height:200%; font-family: 'B Nazanin', serif; font-size: 18px; text-align: justify;">
    <h3>
        4- انتخاب زیرمجموعه دیتاست و آماده‌سازی پوشه خروجی
    </h3>
    <p>
    <div dir="rtl" style="text-align: justify;">
        در این سلول، محدوده‌ای از ایندکس‌های دیتاست پردازش شده انتخاب شده و زیرمجموعه‌ای از دیتاست مربوط به بخش 'train' استخراج می‌شود. همچنین پوشه‌ای جهت ذخیره‌سازی فایل‌های صوتی بدون نویز ایجاد می‌شود.
    </div>
    </p>
</div>

In [ ]:
import os
import torch
import torchaudio
import noisereduce as nr
import numpy as np
from tqdm import tqdm

# Define the range of indices to process
start_idx = 0
end_idx = 113203

# Specify the split to process
split = 'train'  

# Access the specific split and create a subset of the main dataset
data_chunk = dataset_pp[split].select(range(start_idx, end_idx + 1))

# Output folder path
output_dir = f"denoised_audios_{split}"
os.makedirs(output_dir, exist_ok=True)  # Ensure the output folder exists

<div style="direction:rtl; line-height:200%; font-family: 'B Nazanin', serif; font-size: 18px; text-align: justify;">
    <h3>
        5- حذف نویز و ذخیره‌سازی صوت‌های پاک شده
    </h3>
    <p>
    <div dir="rtl" style="text-align: justify;">
        در این سلول، به ازای هر فایل صوتی موجود در زیرمجموعه دیتاست، ابتدا نویز کاهش داده می‌شود، سپس سیگنال نرمال شده و به فرمت int16 تبدیل می‌شود تا در نهایت به صورت فایل MP3 ذخیره شود. در صورت بروز خطا، پیام خطا چاپ شده و پردازش ادامه می‌یابد.
    </div>
    </p>
    <p>
    <div dir="rtl" style="text-align: justify;">
        لازم به ذکر است که سه فایل صوتی با ایندکس‌های 27615، 72585 و 79264 به دلیل خطا (عدم امکان دسترسی یا وجود فایل) به درستی پردازش و ذخیره نشده‌اند. و آن ها را از دیتاست train حذف کردیم 
    </div>
    </p>
</div>

In [ ]:
for i in tqdm(range(len(data_chunk)), desc=f"Processing {split} audios"):
    try:
        # Get audio data
        signal = data_chunk[i]["audio"]["array"]
        sr = data_chunk[i]["audio"]["sampling_rate"]

        # Noise reduction
        denoised_signal = nr.reduce_noise(y=signal, sr=sr)

        # Normalize the signal to prevent clipping
        if np.max(np.abs(denoised_signal)) > 0:
            denoised_signal = denoised_signal / np.max(np.abs(denoised_signal))

        # Convert signal to int16 for saving as MP3 format
        denoised_signal_int16 = np.int16(denoised_signal * 32767)

        # Output file name with index format
        file_idx = start_idx + i
        filename = f"audio_{file_idx}.mp3"
        file_path = os.path.join(output_dir, filename)

        # Convert array to tensor and add channel dimension
        tensor_signal = torch.from_numpy(denoised_signal_int16).unsqueeze(0)

        # Save the MP3 file
        torchaudio.save(file_path, tensor_signal, sr, format="mp3")

    except Exception as e:
        print(f"Error processing file index {start_idx + i}: {e}")
        continue

print(f"All processed audio files have been saved in the folder '{output_dir}' in MP3 format.")

Processing train audios:  24%|██▍       | 27617/113204 [1:14:26<2:40:46,  8.87it/s]

Error processing file index 27615: Error opening <_io.BytesIO object at 0x797418bfd7b0>: File does not exist or is not a regular file (possibly a pipe?).


Processing train audios:  25%|██▌       | 28481/113204 [1:16:54<5:03:03,  4.66it/s]/usr/local/lib/python3.11/dist-packages/noisereduce/spectralgate/nonstationary.py:70: RuntimeWarning: invalid value encountered in divide
  sig_mult_above_thresh = (abs_sig_stft - sig_stft_smooth) / sig_stft_smooth
<ipython-input-9-3d585f93de03>:15: RuntimeWarning: invalid value encountered in cast
  denoised_signal_int16 = np.int16(denoised_signal * 32767)
Processing train audios:  64%|██████▍   | 72587/113204 [3:17:11<1:17:52,  8.69it/s]

Error processing file index 72585: Error opening <_io.BytesIO object at 0x797418bffdd0>: File does not exist or is not a regular file (possibly a pipe?).


Processing train audios:  70%|███████   | 79266/113204 [3:35:04<54:45, 10.33it/s]  

Error processing file index 79264: Error opening <_io.BytesIO object at 0x797418c37ec0>: File does not exist or is not a regular file (possibly a pipe?).


Processing train audios: 100%|██████████| 113204/113204 [5:05:30<00:00,  6.18it/s]

All processed audio files have been saved in the folder 'denoised_audios_train' in MP3 format.


<div style="direction:rtl; line-height:200%; font-family: 'B Nazanin', serif; font-size: 18px; text-align: justify;">
    <h3>
        6- فشرده‌سازی پوشه و انتقال آن به Google Drive
    </h3>
    <p>
    <div dir="rtl" style="text-align: justify;">
        در این سلول، پوشه‌ی denoised_audios_train به صورت یک فایل ZIP فشرده می‌شود. سپس، Google Drive به وسیله‌ی Colab mount شده و فایل ZIP به پوشه‌ی مشخصی در Drive کپی می‌شود. در پایان، با لیست کردن محتویات پوشه‌ی مقصد، صحت انتقال فایل تایید می‌شود.
    </div>
    </p>
</div>

In [ ]:
import os
import shutil
from google.colab import drive
from tqdm import tqdm
import zipfile

# Define the folder to zip and the name of the zip file
folder_to_zip = "denoised_audios_train"
zip_filename = "denoised_audios_train.zip"

# Check if the folder exists
if not os.path.exists(folder_to_zip):
    print(f"The folder '{folder_to_zip}' does not exist. Please check the folder name and try again.")
else:
    # Create a zip archive of the folder
    print(f"Zipping the folder '{folder_to_zip}' into '{zip_filename}'...")

    # Initialize the zip file
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through all files and directories in the folder
        for root, dirs, files in tqdm(os.walk(folder_to_zip), desc="Zipping files"):
            for file in files:
                file_path = os.path.join(root, file)
                # Add file to the zip archive with relative path
                zipf.write(file_path, os.path.relpath(file_path, folder_to_zip))

    print(f"Folder '{folder_to_zip}' has been zipped successfully as '{zip_filename}'.")

    # Mount Google Drive
    print("Mounting Google Drive...")
    drive.mount('/content/drive')

    # Define the destination path in Google Drive
    drive_folder = 'ProcessedAudios'
    drive_path = f'/content/drive/MyDrive/{drive_folder}'

    # Create the destination folder in Drive if it doesn't exist
    os.makedirs(drive_path, exist_ok=True)
    drive_destination = os.path.join(drive_path, zip_filename)

    # Verify if the zip file exists before uploading
    if os.path.exists(zip_filename):
        print(f"Uploading '{zip_filename}' to Google Drive at '{drive_destination}'...")
        shutil.copy(zip_filename, drive_destination)
        print(f"'{zip_filename}' has been successfully uploaded to Google Drive in folder '{drive_folder}'.")
    else:
        print(f"Zip file '{zip_filename}' not found. Please ensure the zipping process was successful.")

    # Verify the upload by listing files in the destination folder
    print("Verifying the uploaded files in Google Drive:")
    uploaded_files = os.listdir(drive_path)
    print(uploaded_files)

Zipping the folder 'denoised_audios_train' into 'denoised_audios_train.zip'...


Zipping files: 1it [03:07, 187.67s/it]


Folder 'denoised_audios_train' has been zipped successfully as 'denoised_audios_train.zip'.
Mounting Google Drive...
Mounted at /content/drive
Uploading 'denoised_audios_train.zip' to Google Drive at '/content/drive/MyDrive/ProcessedAudios/denoised_audios_train.zip'...
'denoised_audios_train.zip' has been successfully uploaded to Google Drive in folder 'ProcessedAudios'.
Verifying the uploaded files in Google Drive:
['denoised_audios_train.zip']
